In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sps
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

# CDMX

In [2]:
cd = pd.read_csv('listings - CDMX_clean.csv')
cd.drop(columns=['Unnamed: 0'], inplace=True)
cd

,id,host_id,beds,bedrooms,source,name,description,neighborhood_overview,host_name,host_since,...,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,44616,196253,1.0,5.0,previous scrape,Condesa Haus,A new concept of hosting in mexico through a b...,No neighborhood_overview,Fernando,2010-08-09,...,0.0,0.0,4.58,4.56,4.70,4.87,4.78,4.98,4.48,0.41
1,67703,334451,2.0,2.0,city scrape,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",No neighborhood_overview,Nicholas,2011-01-04,...,4.0,0.0,4.90,4.81,4.75,4.94,4.92,4.98,4.91,0.31
2,70644,212109,1.0,1.0,city scrape,Beautiful light Studio Coyoacan- full equipped !,COYOACAN designer studio quiet & safe! well eq...,Coyoacan is a beautiful neighborhood famous fo...,Trisha,2010-08-24,...,7.0,2.0,4.91,4.90,4.96,4.96,4.98,4.96,4.92,0.83
3,107078,540705,1.0,1.0,city scrape,NEW DESIGNER LOFT,Is the best ever place triple L <br />Location...,"Is located in the best area of Mexico City, Po...",Andrea,2011-04-27,...,0.0,0.0,4.91,5.00,5.00,5.00,4.73,4.91,4.82,0.11
4,131610,647454,1.0,2.0,previous scrape,MARIA DEL ALMA,No description,No neighborhood_overview,Fernando,2011-05-31,...,0.0,0.0,4.80,4.80,4.80,4.90,4.90,4.90,4.80,1.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26576,1252141718111439700,67472234,2.0,2.0,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,Experience Mexico City in this delightful retr...,"Welcome to Lomas de Bezares, a neighborhood kn...",HostPal,2016-04-16,...,0.0,0.0,4.80,4.80,4.80,4.90,4.90,4.90,4.80,1.00
26577,1252159821531694005,596009321,1.0,1.0,city scrape,Suite King | District Polanco,This spacious room features a plush king-size ...,No neighborhood_overview,Luis Angel,2024-08-13,...,0.0,0.0,4.80,4.80,4.80,4.90,4.90,4.90,4.80,1.00
26578,1252185591721328995,596009321,1.0,1.0,city scrape,Suite King | District Polanco,Discover the essence of luxury in our king-siz...,No neighborhood_overview,Luis Angel,2024-08-13,...,0.0,0.0,4.80,4.80,4.80,4.90,4.90,4.90,4.80,1.00
26579,1252218347521498445,596009321,1.0,1.0,city scrape,Suite Queen | District Polanco,Welcome to your refuge in the heart of Polanco...,No neighborhood_overview,Luis Angel,2024-08-13,...,0.0,0.0,4.80,4.80,4.80,4.90,4.90,4.90,4.80,1.00


In [3]:
# Transformamos algunas variables para su tratamiento

# Transformamos host_response_rate a float
# Los No data los quitamos
cd = cd[cd['host_acceptance_rate'] != 'No data']
cd['host_acceptance_rate'] = cd['host_acceptance_rate'].str.replace('%', '').astype(float)

# Pasamos host_is_superhost a entero, donde t es 1 y f es 0
cd['host_is_superhost'] = cd['host_is_superhost'].replace({'t': 1, 'f': 0})
cd['host_is_superhost'] = cd['host_is_superhost'].astype(int)

# Dado que no tenemos la variable bathrooms, la creamos usando bathrooms_text
# Quitaremos el texto y nos quedaremos solo con el número de la izquierda
cd['bathrooms'] = cd['bathrooms_text'].str.extract('(\d+\.?\d*)').astype(float)
# Quitamos los valores nulos
cd = cd[cd['bathrooms'].notnull()]

# Transformamos las variable instant_bookable a entero, donde t es 1 y f es 0
cd['instant_bookable'] = cd['instant_bookable'].replace({'t': 1, 'f': 0})

# Transformamos la variable host_identity_verified a entero, donde t es 1 y f es 0
cd['host_identity_verified'] = cd['host_identity_verified'].replace({'t': 1, 'f': 0})

# Transformamos la variable has_availability a entero, donde t es 1 y f es 0
cd['has_availability'] = cd['has_availability'].replace({'t': 1, 'f': 0})

# Transformamos a numerico la variable host_response_time
# Donde:
types = {
    'within an hour': 1,
    'within a few hours': 2,
    'within a day': 3,
    'No data': 0,
    'a few days or more': 4
}
cd['host_response_time'] = cd['host_response_time'].replace(types).astype(int)

# Hacemos algo parecido con room_type
# Donde:
types = {
    'Entire home/apt': 1,
    'Private room': 2,
    'Shared room': 3,
    'Hotel room': 4,
}
cd['room_type'] = cd['room_type'].replace(types).astype(int)

C:\Users\angel\AppData\Local\Temp\ipykernel_23724\4001365125.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cd['host_acceptance_rate'] = cd['host_acceptance_rate'].str.replace('%', '').astype(float)
C:\Users\angel\AppData\Local\Temp\ipykernel_23724\4001365125.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cd['host_is_superhost'] = cd['host_is_superhost'].replace({'t': 1, 'f': 0})
C:\Users\angel\AppData\Local\Temp\ipykernel_23724\4001365125.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [4]:
# Usaremos el mismo escalar para cuando sea necesario
escalar = StandardScaler()

## host_is_superhost

In [5]:
# Vamos a usar la regresión logística para predecir si un host es superhost o no

X = cd[['price', 'number_of_reviews', 'host_acceptance_rate']]
y = cd['host_is_superhost']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[2364  458]
 [1168  740]] 

Precisión: 0.6176961602671118
Exactitud: 0.6562367864693446
Sensibilidad: 0.38784067085953877


## instant_bookable

In [6]:
# Predeciremos si un alojamiento puede reservarse instantáneamente o no

X = cd[['price', 'number_of_reviews', 'host_response_time']]
y = cd['instant_bookable']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[2243  479]
 [1521  487]] 

Precisión: 0.5041407867494824
Exactitud: 0.5771670190274841
Sensibilidad: 0.24252988047808766


## has_availability

In [7]:
# Predeciremos si un alojamiento está disponible o no

X = cd[['availability_365', 'number_of_reviews', 'host_response_time']]
y = cd['has_availability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[   0   26]
 [   0 4704]] 

Precisión: 0.9945031712473573
Exactitud: 0.9945031712473573
Sensibilidad: 1.0


## room_type(Entire home/apt)

In [8]:
# Vamos a tratar de predecir si un alojamiento es una casa completa o no
# Temporalmente, convertiremos la variable room_type a binaria, donde 1 es casa completa y 0 es el resto
ny_temp = cd.copy()
ny_temp['room_type'] = ny_temp['room_type'].replace({1: 1, 2: 0, 3: 0, 4: 0}).astype(int)

X = ny_temp[['price', 'accommodates', 'beds']]
y = ny_temp['room_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[1088  479]
 [ 396 2767]] 

Precisión: 0.8524337646333949
Exactitud: 0.8150105708245243
Sensibilidad: 0.8748024027821688


## room_type(Private room)

In [9]:
# Predeciremos si un alojamiento es una habitación privada o no
# Temporalmente, convertiremos la variable room_type a binaria, donde 1 es habitación privada y 0 es el resto
ny_temp = cd.copy()
ny_temp['room_type'] = ny_temp['room_type'].replace({1: 0, 2: 1, 3: 0, 4: 0}).astype(int)

X = ny_temp[['price', 'accommodates', 'beds']]
y = ny_temp['room_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)


algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[2857  396]
 [ 485  992]] 

Precisión: 0.7146974063400576
Exactitud: 0.8137420718816067
Sensibilidad: 0.6716316858496953


## host_response_time(within an hour)

In [10]:
# Predeciremos si un host responde en menos de una hora o no
# Temporalmente, convertiremos la variable host_response_time a binaria, donde 1 es menos de una hora y 0 es el resto
ny_temp = cd.copy()
ny_temp['host_response_time'] = ny_temp['host_response_time'].replace({1: 1, 2: 0, 3: 0, 4: 0}).astype(int)

X = ny_temp[['host_is_superhost', 'instant_bookable', 'review_scores_communication']]
y = ny_temp['host_response_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[   0  997]
 [   0 3733]] 

Precisión: 0.7892177589852009
Exactitud: 0.7892177589852009
Sensibilidad: 1.0


## host_response_time(within a day)

In [11]:
# Predeciremos si un host responde en menos de un día o no
# Temporalmente, convertiremos la variable host_response_time a binaria, donde 1 es menos de un día y 0 es el resto
ny_temp = cd.copy()
ny_temp['host_response_time'] = ny_temp['host_response_time'].replace({1: 0, 2: 0, 3: 1, 4: 0}).astype(int)

X = ny_temp[['host_is_superhost', 'instant_bookable', 'review_scores_communication']]
y = ny_temp['host_response_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[4527    0]
 [ 203    0]] 

Precisión: 0.0
Exactitud: 0.9570824524312896
Sensibilidad: 0.0


c:\Users\angel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## host_identity_verified

In [12]:
# Predeciremos si un host está verificado o no

X = cd[['host_is_superhost', 'instant_bookable', 'number_of_reviews']]
y = cd['host_identity_verified']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[   0   78]
 [   0 4652]] 

Precisión: 0.9835095137420719
Exactitud: 0.9835095137420719
Sensibilidad: 1.0


## review_scores_rating

In [13]:
# Vamos a predecir si las reseñas son positivas o no con review_scores_rating
# Temporalmente, convertiremos la variable review_scores_rating a binaria, donde será buena si es mayor o igual a 4.5
ny_temp = cd.copy()
ny_temp['review_scores_rating'] = (ny_temp['review_scores_rating'] >= 4.5).astype(int)

X = ny_temp[['review_scores_cleanliness', 'review_scores_communication', 'review_scores_location']]
y = ny_temp['review_scores_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[  98  243]
 [  37 4352]] 

Precisión: 0.9471164309031556
Exactitud: 0.9408033826638478
Sensibilidad: 0.9915698336750969


## reviews_per_month

In [14]:
# Verificaremos si un lugar es constantemente visitado o no con reviews_per_month
# Temporalmente, convertiremos la variable reviews_per_month a binaria, donde será 1 si es mayor a 1
ny_temp = cd.copy()
ny_temp['reviews_per_month'] = (ny_temp['reviews_per_month'] > 1).astype(int)

X = ny_temp[['room_type', 'price', 'review_scores_rating']]
y = ny_temp['reviews_per_month']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(matriz, "\n")
print("Precisión:", precision_score(y_test, y_pred))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad:", recall_score(y_test, y_pred))

Matriz de confusión:
[[ 894 1118]
 [ 676 2042]] 

Precisión: 0.6462025316455696
Exactitud: 0.6207188160676532
Sensibilidad: 0.7512877115526122
